## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Butaritari,KI,3.0707,172.7902,81.10,74,72,15.79,broken clouds
1,1,Lagoa,PT,39.0500,-27.9833,58.21,67,94,11.41,overcast clouds
2,2,Oranjemund,NaN,-28.5500,16.4333,78.04,60,71,8.97,broken clouds
3,3,Rikitea,PF,-23.1203,-134.9692,77.88,72,100,11.05,overcast clouds
4,4,Mataura,NZ,-46.1927,168.8643,54.48,88,32,5.01,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Butaritari,KI,3.0707,172.7902,81.10,74,72,15.79,broken clouds
2,2,Oranjemund,NaN,-28.5500,16.4333,78.04,60,71,8.97,broken clouds
3,3,Rikitea,PF,-23.1203,-134.9692,77.88,72,100,11.05,overcast clouds
6,6,Alice Springs,AU,-23.7000,133.8833,80.15,74,75,5.75,thunderstorm
9,9,Kavaratti,IN,10.5669,72.6420,79.84,68,87,9.17,overcast clouds
10,10,Merauke,ID,-8.4667,140.3333,77.05,93,99,8.03,overcast clouds
13,13,Sao Joao Da Barra,BR,-21.6403,-41.0511,84.29,70,2,8.21,clear sky
15,15,Georgetown,MY,5.4112,100.3354,83.89,75,20,3.00,few clouds
16,16,Hithadhoo,MV,-0.6000,73.0833,82.74,70,37,13.98,scattered clouds
27,27,Arlit,NE,18.7369,7.3853,78.76,9,6,4.05,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                192
City                   192
Country                186
Lat                    192
Lng                    192
Max Temp               192
Humidity               192
Cloudiness             192
Wind Speed             192
Current Description    192
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                186
City                   186
Country                186
Lat                    186
Lng                    186
Max Temp               186
Humidity               186
Cloudiness             186
Wind Speed             186
Current Description    186
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Butaritari,KI,81.10,broken clouds,3.0707,172.7902,
3,Rikitea,PF,77.88,overcast clouds,-23.1203,-134.9692,
6,Alice Springs,AU,80.15,thunderstorm,-23.7000,133.8833,
9,Kavaratti,IN,79.84,overcast clouds,10.5669,72.6420,
10,Merauke,ID,77.05,overcast clouds,-8.4667,140.3333,
13,Sao Joao Da Barra,BR,84.29,clear sky,-21.6403,-41.0511,
15,Georgetown,MY,83.89,few clouds,5.4112,100.3354,
16,Hithadhoo,MV,82.74,scattered clouds,-0.6000,73.0833,
27,Arlit,NE,78.76,clear sky,18.7369,7.3853,
30,Acarau,BR,85.30,scattered clouds,-2.8856,-40.1200,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# Hotel Names with empty strings
hotel_df.loc[hotel_df["Hotel Name"] == ''].count().iloc[0]

17

In [10]:
hotel_df["Hotel Name"].replace('',np.nan,inplace=True)

In [20]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Butaritari,KI,81.10,broken clouds,3.0707,172.7902,Isles Sunset Lodge
3,Rikitea,PF,77.88,overcast clouds,-23.1203,-134.9692,People ThankYou
6,Alice Springs,AU,80.15,thunderstorm,-23.7000,133.8833,Desert Palms Alice Springs
9,Kavaratti,IN,79.84,overcast clouds,10.5669,72.6420,Hotels in Lakshadweep Islands
13,Sao Joao Da Barra,BR,84.29,clear sky,-21.6403,-41.0511,Pousada Porto De Canoas
...,...,...,...,...,...,...,...
555,Iracoubo,GF,80.13,broken clouds,5.4802,-53.2011,Le Gîte d’Iracoubo
559,Nouadhibou,MR,75.18,broken clouds,20.9310,-17.0347,El Medina
560,Port-Gentil,GA,84.16,few clouds,-0.7193,8.7815,Fengshui Residence
563,Eyl,SO,77.20,overcast clouds,7.9803,49.8164,Amiin Hotel


In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()

In [13]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Butaritari,KI,81.10,broken clouds,3.0707,172.7902,Isles Sunset Lodge
3,Rikitea,PF,77.88,overcast clouds,-23.1203,-134.9692,People ThankYou
6,Alice Springs,AU,80.15,thunderstorm,-23.7000,133.8833,Desert Palms Alice Springs
9,Kavaratti,IN,79.84,overcast clouds,10.5669,72.6420,Hotels in Lakshadweep Islands
13,Sao Joao Da Barra,BR,84.29,clear sky,-21.6403,-41.0511,Pousada Porto De Canoas
...,...,...,...,...,...,...,...
555,Iracoubo,GF,80.13,broken clouds,5.4802,-53.2011,Le Gîte d’Iracoubo
559,Nouadhibou,MR,75.18,broken clouds,20.9310,-17.0347,El Medina
560,Port-Gentil,GA,84.16,few clouds,-0.7193,8.7815,Fengshui Residence
563,Eyl,SO,77.20,overcast clouds,7.9803,49.8164,Amiin Hotel


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))